# PLANTILLA TRANSFORMACIÓN DE DATOS

**IMPORTANTE**: Recuerda hacer una copia de esta plantilla para no machacar la original.

## IMPORTAR PAQUETES

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

C:\Users\acrespod\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [1]:
ruta_proyecto = 'C:/proyectos/weather_ohio/'

2.- Nombrar los ficheros de datos.

In [2]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [74]:
# cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat).reset_index(drop = False)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)
df = num.copy()


4.- Separar la target.

In [12]:
# target = num[['contrata_fondos']].copy().reset_index(drop=True)

# NUEVAS VARIABLES

- extraccion componentes temporales: df_temp
- variables lags
- target

In [75]:
df

,awnd,prcp,tmax,tmin,wdf2,wdf5,wsf2,wsf5
DATE,,,,,,,,
1998-03-26,11.63,0.23,25.000000,8.888889,250.0,250.0,29.1,36.0
1998-03-27,11.63,0.23,26.666667,17.777778,250.0,250.0,29.1,36.0
1998-03-28,11.63,0.23,25.555556,16.111111,250.0,250.0,29.1,36.0
1998-03-29,11.63,0.23,26.111111,15.000000,250.0,250.0,29.1,36.0
1998-04-01,11.63,0.23,21.666667,10.000000,250.0,250.0,29.1,36.0
...,...,...,...,...,...,...,...,...
2021-12-27,9.62,0.40,15.000000,1.666667,80.0,280.0,21.9,30.0
2021-12-28,7.38,0.29,11.666667,3.333333,300.0,300.0,21.0,29.1
2021-12-29,4.03,0.05,10.000000,6.111111,230.0,220.0,13.0,16.1


## Extraccion temporal

In [133]:
def extraccion_componentes_temporales(df):
    
    year =  df.DATE.dt.year
    month = df.DATE.dt.month
    day = df.DATE.dt.dayofyear
    year_month = df.DATE.dt.to_period("M")
    
    year.name = 'year'
    month.name = 'month'
    day.name = 'day'
    year_month.name = 'year_month'
    
    temp = pd.concat([year,month,day,year_month],axis=1)
    temp.index = df.DATE
    
    return temp

df_temp = extraccion_componentes_temporales(df.reset_index())
df_temp.head()

,year,month,day,year_month
DATE,,,,
1998-03-26,1998,3,85,1998-03
1998-03-27,1998,3,86,1998-03
1998-03-28,1998,3,87,1998-03
1998-03-29,1998,3,88,1998-03
1998-04-01,1998,4,91,1998-04


## tmax_mes _año

In [163]:
#mes

In [198]:
# tmax_mensual: pone una tmax media de un mes N en todos los registros del mes n+1
df1 = pd.concat([df,df_temp],axis=1)
df2 = df1[['year_month','month','year','tmax']]
df3 = df2.groupby(['year_month'],as_index=False).apply(lambda x: x.tmax.mean() )

df3.columns = ['year_month','tmax_mensual']
df3.set_index('year_month',inplace=True)
df3a = df3.shift(1).reset_index()
df4 = df_temp.reset_index().merge(df3a, how='left',left_on='year_month' ,right_on='year_month')
df4

,DATE,year,month,day,year_month,tmax_mensual
0,1998-03-26,1998,3,85,1998-03,NaN
1,1998-03-27,1998,3,86,1998-03,NaN
2,1998-03-28,1998,3,87,1998-03,NaN
3,1998-03-29,1998,3,88,1998-03,NaN
4,1998-04-01,1998,4,91,1998-04,25.833333
...,...,...,...,...,...,...
8667,2021-12-27,2021,12,361,2021-12,10.314815
8668,2021-12-28,2021,12,362,2021-12,10.314815
8669,2021-12-29,2021,12,363,2021-12,10.314815
8670,2021-12-30,2021,12,364,2021-12,10.314815


In [204]:
# tmax_anual: tmax media de los registro de n mes , se pone todo los registros del mismo mes el año siguiente
df3b = df3.shift(12).reset_index()
df4b = df_temp.reset_index().merge(df3b, how='left',left_on='year_month' ,right_on='year_month')
df4b

,DATE,year,month,day,year_month,tmax_mensual
0,1998-03-26,1998,3,85,1998-03,NaN
1,1998-03-27,1998,3,86,1998-03,NaN
2,1998-03-28,1998,3,87,1998-03,NaN
3,1998-03-29,1998,3,88,1998-03,NaN
4,1998-04-01,1998,4,91,1998-04,NaN
...,...,...,...,...,...,...
8667,2021-12-27,2021,12,361,2021-12,4.641577
8668,2021-12-28,2021,12,362,2021-12,4.641577
8669,2021-12-29,2021,12,363,2021-12,4.641577
8670,2021-12-30,2021,12,364,2021-12,4.641577


## variables lag

In [49]:
df2 = df.loc[:'1998-04-03'].copy()
df2

,prcp_attributes,awnd,prcp,tmax,tmin,wdf2,wdf5,wsf2,wsf5
DATE,,,,,,,,,
1998-03-26,",,W,2400",11.63,0.23,25.000000,8.888889,250.0,250.0,29.1,36.0
1998-03-27,",,W,2400",11.63,0.23,26.666667,17.777778,250.0,250.0,29.1,36.0
1998-03-28,",,W,2400",11.63,0.23,25.555556,16.111111,250.0,250.0,29.1,36.0
1998-03-29,",,W,2400",11.63,0.23,26.111111,15.000000,250.0,250.0,29.1,36.0
1998-04-01,",,W,2400",11.63,0.23,21.666667,10.000000,250.0,250.0,29.1,36.0
1998-04-02,",,W,2400",11.41,0.00,11.666667,6.666667,250.0,260.0,25.1,33.1
1998-04-03,"T,,W,2400",5.59,0.00,12.222222,1.666667,30.0,40.0,17.0,21.0


In [98]:
def lag_variable(variable, n=10):
    lista = []
    temp = pd.DataFrame()
    for i in range(1,n+1):
        temp[variable.name +f"_{i}"] = variable.shift(i)
    
    return temp
        
dias_lag=7    
df_awnd = lag_variable(df.awnd,n= dias_lag)
df_prcp = lag_variable(df.prcp,n= dias_lag)
df_tmin = lag_variable(df.tmin,n= dias_lag)
df_wdf2 = lag_variable(df.wdf2,n= dias_lag)
df_wdf5 = lag_variable(df.wdf5,n= dias_lag)
df_wsf2 = lag_variable(df.wsf2,n= dias_lag)
df_wsf5 = lag_variable(df.wsf5,n= dias_lag)

df_tmax = lag_variable(df.tmax,n= dias_lag)


## rolling variables

In [91]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    #Devuelve el dataframe de lags
    return(minm)

###################################

def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    #Devuelve el dataframe de lags
    return(mm)

######################################

def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()
    
    #Devuelve el dataframe de lags
    return(maxm)

In [99]:
df_tmax_maxm = df.transform(lambda x: max_movil(x,'tmax',num_periodos =7))
df_tmax_minm = df.transform(lambda x: min_movil(x,'tmax',num_periodos =7))
df_tmax_mm = df.transform(lambda x: media_movil(x,'tmax',num_periodos =7))




## Union

In [272]:
# no he metido tmax_anual por ahora, quitaria un año de registros
df_lag = pd.concat([df_awnd, df_prcp, df_tmin, df_wdf2, df_wdf5, df_wsf2, df_wsf5, df_tmax],axis=1)
df_moviles = pd.concat([df_tmax_maxm,df_tmax_minm, df_tmax_mm ],axis=1)
df_target = df[['tmax']].rename(columns = {'tmax':'target'})

df_final = pd.concat([df4.set_index('DATE'),df_lag,df_moviles,df_target],axis=1)
df_final.shape

(8672, 80)

In [273]:
df_final.dropna(inplace=True)

In [274]:
df_final

,year,month,day,year_month,tmax_mensual,awnd_1,awnd_2,awnd_3,awnd_4,awnd_5,...,tmax_minm_5,tmax_minm_6,tmax_minm_7,tmax_mm_2,tmax_mm_3,tmax_mm_4,tmax_mm_5,tmax_mm_6,tmax_mm_7,target
DATE,,,,,,,,,,,,,,,,,,,,,
1998-04-04,1998,4,94,1998-04,25.833333,5.590,11.410,11.63,11.63,11.63,...,11.666667,11.666667,11.666667,11.944444,15.185185,17.916667,19.444444,20.648148,21.269841,8.333333
1998-04-05,1998,4,95,1998-04,25.833333,14.990,5.590,11.41,11.63,11.63,...,8.333333,8.333333,8.333333,10.277778,10.740741,13.472222,16.000000,17.592593,18.888889,12.222222
1998-04-07,1998,4,97,1998-04,25.833333,7.830,14.990,5.59,11.41,11.63,...,8.333333,8.333333,8.333333,10.277778,10.925926,11.111111,13.222222,15.370370,16.825397,16.666667
1998-04-08,1998,4,98,1998-04,25.833333,8.055,7.830,14.99,5.59,11.41,...,8.333333,8.333333,8.333333,14.444444,12.407407,12.361111,12.222222,13.796296,15.555556,21.111111
1998-04-09,1998,4,99,1998-04,25.833333,8.280,8.055,7.83,14.99,5.59,...,8.333333,8.333333,8.333333,18.888889,16.666667,14.583333,14.111111,13.703704,14.841270,17.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,2021,12,361,2021-12,10.314815,6.260,12.080,11.41,8.28,10.51,...,2.222222,2.222222,2.222222,12.777778,13.333333,12.361111,10.333333,9.814815,9.126984,15.000000
2021-12-28,2021,12,362,2021-12,10.314815,9.620,6.260,12.08,11.41,8.28,...,9.444444,2.222222,2.222222,12.777778,13.518519,13.750000,12.888889,11.111111,10.555556,11.666667
2021-12-29,2021,12,363,2021-12,10.314815,7.380,9.620,6.26,12.08,11.41,...,10.555556,9.444444,2.222222,13.333333,12.407407,13.055556,13.333333,12.685185,11.190476,10.000000


## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [30]:
# var_ohe = ['estado_civil','hipoteca','impago','consumo']

#### Instanciar

In [31]:
# ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [32]:
# cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [33]:
# cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Ordinal Encoding

#### Variables a aplicar OE

In [34]:
# var_oe = ['formacion','resultado_campana_anterior']

#### Orden de los valores de las variables

In [35]:
#Orden de la primera variable
# orden_formacion = ['NULO','illiterate','basic.4y','basic.6y','basic.9y','high.school','professional.course','university.degree']

# #Orden de la segunda variable
# orden_resultado_campana_anterior = ['nonexistent','failure','success']


#### Instanciar

In [36]:
# oe = OrdinalEncoder(categories = [orden_formacion,orden_resultado_campana_anterior],
#                     handle_unknown = 'use_encoded_value',
#                     unknown_value = 10)


#### Entrenar y aplicar

In [228]:
# cat_oe = oe.fit_transform(cat[var_oe])

#### Guardar como dataframe

In [38]:
# #Añadir sufijos a los nombres
# nombres_oe = [variable + '_oe' for variable in var_oe]

# #Guardar como dataframe
# cat_oe = pd.DataFrame(cat_oe, columns = nombres_oe)

### Target Encoding

#### Variables a aplicar TE

In [39]:
# var_te = ['trabajo','mes']

#### Instanciar

In [40]:
# te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Entrenar y aplicar

In [41]:
# cat_te = te.fit_transform(cat[var_te], y = target)

#### Guardar como dataframe

In [42]:
#Añadir sufijos a los nombres
# nombres_te = [variable + '_te' for variable in var_te]

# #Guardar como dataframe
# cat_te = pd.DataFrame(cat_te, columns = nombres_te)

### Fechas y textos

Estas variables requieren transformaciones más ad-hoc y no tan susceptibles de aplicar mediante una plantilla.

Acudir a su correspondientes notebooks para recordar los recursos para trabajarlas.

No obstante esta plantilla contiene esta sección para recordar que aquí sería el momento en el que se trabajarían.

## TRANSFORMACIÓN DE NUMÉRICAS

### Discretizar variables

#### Mediante distancia similar

##### Variables a discretizar mediante distancia similar

In [43]:
# var_disc_ds = ['num_contactos_esta_campana','num_dias_ultimo_contacto']

##### Instanciar

In [44]:
# disc_ds = KBinsDiscretizer(n_bins = 6, strategy = 'uniform', encode = 'ordinal')

##### Entrenar y aplicar

In [45]:
# num_disc_ds = disc_ds.fit_transform(num[var_disc_ds])

##### Guardar como dataframe

In [46]:
#Añadir sufijos a los nombres
# nombres_ds = [variable + '_disc_ds' for variable in var_disc_ds]

# #Guardar como dataframe
# num_disc_ds = pd.DataFrame(num_disc_ds,columns = nombres_ds)

#### Mediante número de casos similares

##### Variables a discretizar mediante número de casos similares

In [47]:
# var_disc_cs = ['num_contactos_esta_campana','euribor3m']

##### Instanciar

In [48]:
# disc_cs = KBinsDiscretizer(n_bins = 2, strategy = 'quantile', encode = 'ordinal')

##### Entrenar y aplicar

In [49]:
# num_disc_cs = disc_cs.fit_transform(num[var_disc_cs])

##### Guardar como dataframe

In [50]:
#Añadir sufijos a los nombres
# nombres_cs = [variable + '_disc_cs' for variable in var_disc_cs]

# #Guardar como dataframe
# num_disc_cs = pd.DataFrame(num_disc_cs,columns = nombres_cs)

### Binarizar variables

#### Variables a binarizar

In [51]:
# var_bin = ['variacion_tasa_empleo']

#### Instanciar

In [52]:
# bin = Binarizer(threshold=0)

#### Entrenar y aplicar

In [53]:
# num_bin = bin.fit_transform(num[var_bin])

#### Guardar como dataframe

In [54]:
# #Añadir sufijos a los nombres
# nombres_bin = [variable + '_bin' for variable in var_bin]

# #Guardar como dataframe
# num_bin = pd.DataFrame(num_bin,columns = nombres_bin)

### Normalizar (Gauss)

#### Con Yeo-Johnson

##### Variables a normalizar con Yeo-Johnson

In [229]:
# var_yeo = ['edad','euribor3m']

##### Instanciar

In [56]:
# yeo = PowerTransformer(method = 'yeo-johnson')

##### Entrenar y aplicar

In [57]:
# num_yeo = yeo.fit_transform(num[var_yeo])

##### Guardar como dataframe

In [58]:
# #Añadir sufijos a los nombres
# nombres_yeo = [variable + '_yeo' for variable in var_yeo]

# #Guardar como dataframe
# num_yeo = pd.DataFrame(num_yeo,columns = nombres_yeo)

#### Con Quantile Transformer

##### Variables a normalizar con Quantile Transformer

In [59]:
# var_qt = ['edad','euribor3m']

##### Instanciar

In [60]:
# qt = QuantileTransformer(output_distribution='normal')

##### Entrenar y aplicar

In [61]:
# num_qt = qt.fit_transform(num[var_qt])


##### Guardar como dataframe

In [62]:
# #Añadir sufijos a los nombres
# nombres_qt = [variable + '_qt' for variable in var_qt]

# #Guardar como dataframe
# num_qt = pd.DataFrame(num_qt,columns = nombres_qt)

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

In [63]:
# dataframes = []
# dataframes.extend(value for name, value in locals().items() if name.startswith('cat_') or name.startswith('num_'))

### Unir todos los dataframes

In [64]:
# df = pd.concat(dataframes, axis = 1)

## REESCALAR VARIABLES

### Con Min-Max

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19330 entries, 0 to 19329
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   estado_civil_OTROS                  19330 non-null  float64
 1   estado_civil_divorced               19330 non-null  float64
 2   estado_civil_married                19330 non-null  float64
 3   estado_civil_single                 19330 non-null  float64
 4   hipoteca_no                         19330 non-null  float64
 5   hipoteca_unknown                    19330 non-null  float64
 6   hipoteca_yes                        19330 non-null  float64
 7   impago_no                           19330 non-null  float64
 8   impago_unknown                      19330 non-null  float64
 9   impago_yes                          19330 non-null  float64
 10  consumo_no                          19330 non-null  float64
 11  consumo_unknown                     19330

#### Variables a reescalar con Min-Max

In [66]:
# var_mms = df.iloc[:,13:].columns

#### Instanciar

In [67]:
# mms = MinMaxScaler()

#### Entrenar y aplicar

In [68]:
# df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [69]:
# #Añadir sufijos a los nombres
# nombres_mms = [variable + '_mms' for variable in var_mms]

# #Guardar como dataframe
# df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

### Con estandarización

#### Variables a reescalar con estandarización

In [259]:
var_ss = df_final.iloc[:,4:-1].columns.tolist()

In [279]:
target = df_final[['target']].copy()
df_temp2 = df_final.iloc[:,:4]

In [282]:
len(var_ss), df_final.shape[1], target.shape[1], df_temp2.shape[1]

(75, 80, 1, 4)

In [ ]:
[df_temp2,df_ss,target]

#### Instanciar

In [284]:
ss = StandardScaler()

#### Entrenar y aplicar

In [285]:
df_final.loc[:,var_ss].shape

(8665, 75)

In [286]:
df_ss = ss.fit_transform(df_final[var_ss])

In [287]:
df_ss.shape

(8665, 75)

#### Guardar como dataframe

In [288]:
#Añadir sufijos a los nombres
nombres_ss = [variable + '_ss' for variable in var_ss]

#Guardar como dataframe
df_ss = pd.DataFrame(df_ss,columns = nombres_ss, index=df_final.index)

In [289]:
df_ss

,tmax_mensual_ss,awnd_1_ss,awnd_2_ss,awnd_3_ss,awnd_4_ss,awnd_5_ss,awnd_6_ss,awnd_7_ss,prcp_1_ss,prcp_2_ss,...,tmax_minm_4_ss,tmax_minm_5_ss,tmax_minm_6_ss,tmax_minm_7_ss,tmax_mm_2_ss,tmax_mm_3_ss,tmax_mm_4_ss,tmax_mm_5_ss,tmax_mm_6_ss,tmax_mm_7_ss
DATE,,,,,,,,,,,,,,,,,,,,,
1998-04-04,0.988020,-0.597336,1.107255,1.171422,1.171186,1.171049,1.170785,1.170823,-0.438934,-0.438911,...,-0.101562,-0.039324,0.011108,0.053251,-0.451473,-0.141700,0.127958,0.282674,0.406791,0.472802
1998-04-05,0.988020,2.156195,-0.597513,1.106982,1.171186,1.171049,1.170785,1.170823,-0.438934,-0.438911,...,-0.403276,-0.340792,-0.290463,-0.248529,-0.612103,-0.577815,-0.313997,-0.063337,0.097357,0.230082
1998-04-07,0.988020,0.058825,2.155893,-0.597761,1.106751,1.171049,1.170785,1.170823,-0.438934,-0.438911,...,-0.403276,-0.340792,-0.290463,-0.248529,-0.612103,-0.559643,-0.548785,-0.342377,-0.127686,0.019725
1998-04-08,0.988020,0.124734,0.058618,2.155604,-0.597857,1.106617,1.170785,1.170823,-0.438934,-0.438911,...,-0.403276,-0.340792,-0.290463,-0.248529,-0.210528,-0.414272,-0.424485,-0.442832,-0.287091,-0.109725
1998-04-09,0.988020,0.190643,0.124524,0.058360,2.155290,-0.597890,1.106358,1.170823,0.764573,-0.438911,...,-0.403276,-0.340792,-0.290463,-0.248529,0.217819,0.003671,-0.203508,-0.253084,-0.296468,-0.182541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,-0.689596,-0.401073,1.303509,1.106982,0.190011,0.843034,-0.466247,-0.794239,-0.438934,1.796157,...,-0.302705,-0.893482,-0.843343,-0.801791,-0.371158,-0.323415,-0.424485,-0.632580,-0.690293,-0.765068
2021-12-28,-0.689596,0.583167,-0.401259,1.303232,1.106751,0.189932,0.842793,-0.466240,1.280361,-0.438911,...,-0.202134,-0.240303,-0.843343,-0.801791,-0.371158,-0.305243,-0.286374,-0.375862,-0.559018,-0.619436
2021-12-29,-0.689596,-0.072993,0.582937,-0.401510,1.302986,1.106617,0.189737,0.842824,0.807555,1.280372,...,-0.202134,-0.139814,-0.189940,-0.801791,-0.317614,-0.414272,-0.355430,-0.331216,-0.399612,-0.554711


### Con estandarización robusta

#### Variables a reescalar con estandarización robusta

In [74]:
var_rs = df.iloc[:,13:].columns

#### Percentiles a usar para el rango

In [75]:
p_min = 10.0
p_max = 90.0

#### Instanciar

In [76]:
rs = RobustScaler(quantile_range=(p_min, p_max))

#### Entrenar y aplicar

In [77]:
df_rs = rs.fit_transform(df[var_rs])

#### Guardar como dataframe

In [78]:
#Añadir sufijos a los nombres
nombres_rs = [variable + '_rs' for variable in var_rs]

#Guardar como dataframe
df_rs = pd.DataFrame(df_rs,columns = nombres_rs)

## UNIFICAR DATASETS REESCALADOS

### Crear una lista con los dataframes a incluir en el tablón analítico

In [290]:
incluir = [df_temp2,df_ss,target]

### Unir todos los dataframes en el tablón analítico

In [291]:
df_tablon = pd.concat(incluir, axis = 1)

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [292]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [293]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)